In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
import numpy as np
import matplotlib.pyplot as plt

In [2]:
# Define the VAE architecture
class VAE(nn.Module):
    def __init__(self, input_dim, hidden_dim, latent_dim):
        super(VAE, self).__init__()
        
        self.encoder = nn.Sequential(
            nn.Linear(input_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, latent_dim * 2)  # mu and logvar
        )
        
        self.decoder = nn.Sequential(
            nn.Linear(latent_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, input_dim),
            nn.Sigmoid()
        )

    def encode(self, x):
        x = x.view(-1, 28 * 28)  # Flatten MNIST images
        hidden = self.encoder(x)
        mu, logvar = torch.chunk(hidden, 2, dim=1)
        return mu, logvar

    def decode(self, z):
        reconstructed = self.decoder(z)
        return reconstructed

    def reparameterize(self, mu, logvar):
        std = torch.exp(0.5 * logvar)
        eps = torch.randn_like(std)
        return mu + eps * std

    def forward(self, x):
        mu, logvar = self.encode(x)
        z = self.reparameterize(mu, logvar)
        reconstructed = self.decode(z)
        return reconstructed, mu, logvar

In [3]:
# Load MNIST dataset
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('data', train=True, download=True, transform=transform),
    batch_size=128, shuffle=True
)

In [4]:
# Initialize VAE model
input_dim = 28 * 28
hidden_dim = 256
latent_dim = 20
vae = VAE(input_dim, hidden_dim, latent_dim)

# Define optimizer and loss function
optimizer = optim.Adam(vae.parameters(), lr=1e-3)
criterion = nn.BCELoss(reduction='sum')

In [ ]:
# Define L1 regularization function
def l1_regularization(model, lambda_l1):
    l1_reg = torch.tensor(0.0)
    for param in model.parameters():
        l1_reg += torch.norm(param, p=1)
    l1_loss = lambda_l1 * l1_reg
    return l1_loss

# Training loop
def train_vae(model, optimizer, criterion, train_loader, lambda_l1=1.0, num_epochs=20):
    model.train()
    for epoch in range(num_epochs):
        total_loss = 0.0
        for batch_idx, (data, _) in enumerate(train_loader):
            optimizer.zero_grad()
            recon_batch, mu, logvar = model(data)
            loss = criterion(recon_batch, data.view(-1, 784))
            kl_divergence = -0.5 * torch.sum(1 + logvar - mu.pow(2) - logvar.exp())
            loss += kl_divergence
            
            # Add L1 regularization to the loss
            l1_loss = l1_regularization(model, lambda_l1)
            loss += l1_loss

            loss.backward()
            optimizer.step()
            total_loss += loss.item()
        
        print(f"Epoch {epoch+1}/{num_epochs}, Loss: {total_loss / len(train_loader.dataset)}")

# Train the VAE with L1 regularization
train_vae(vae, optimizer, criterion, train_loader)

Epoch 1/20, Loss: -20162.322996858726
Epoch 2/20, Loss: -27639.3388125
Epoch 3/20, Loss: -31487.064425
Epoch 4/20, Loss: -32802.401158333334
Epoch 5/20, Loss: -33553.79168333334
Epoch 6/20, Loss: -34183.1044125
Epoch 7/20, Loss: -35185.09474166667
Epoch 8/20, Loss: -36095.42731666667
Epoch 9/20, Loss: -36948.712583333334
Epoch 10/20, Loss: -37469.04510833333
Epoch 11/20, Loss: -37872.256270833335
Epoch 12/20, Loss: -38151.58511666667
Epoch 13/20, Loss: -38289.9426


In [ ]:
# Latent space attack function
def latent_space_attack(vae, xo, xt, lambda_val=0.1, num_iterations=500):
    vae.eval()
    xo = xo.view(-1, 28 * 28)
    xt = xt.view(-1, 28 * 28)
    
    xo_perturbed = xo.clone().requires_grad_()
    optimizer = torch.optim.Adam([xo_perturbed], lr=1e-3)
    
    for i in range(num_iterations):
        optimizer.zero_grad()
        
        reconstructed_xo, mu_xo, logvar_xo = vae(xo_perturbed)
        reconstructed_xt, mu_xt, logvar_xt = vae(xt)
        
        kl_divergence = torch.sum(0.5 * (logvar_xo.exp() - logvar_xo - 1 + logvar_xt))
        l2_norm = torch.sum((mu_xo - mu_xt) ** 2)  # Using mean difference for closeness
        
        loss = kl_divergence + lambda_val * l2_norm
        
        loss.backward()
        optimizer.step()
        
    return xo_perturbed.detach()

In [ ]:
# Example usage of the attack
target_idx = 0  # Choose a target image index
input_idx = 1  # Choose an input image index

xt, _ = train_loader.dataset[target_idx]
xo, _ = train_loader.dataset[input_idx]

perturbed_xo = latent_space_attack(vae, xo, xt)

In [ ]:
# Visualize original, target, and perturbed images
def show_images(original, target, perturbed):
    fig, axs = plt.subplots(1, 3)
    
    axs[0].imshow(original.reshape(28, 28), cmap='gray')
    axs[0].set_title('Original')
    axs[0].axis('off')
    
    axs[1].imshow(target.reshape(28, 28), cmap='gray')
    axs[1].set_title('Target')
    axs[1].axis('off')
    
    axs[2].imshow(perturbed.reshape(28, 28), cmap='gray')
    axs[2].set_title('Perturbed')
    axs[2].axis('off')
    
    plt.show()

In [ ]:
show_images(xo.squeeze().detach().numpy(), xt.squeeze().detach().numpy(), perturbed_xo.squeeze())

In [ ]:
# Example usage of the attack
target_idx = 0  # Choose a target image index
input_idx = 1  # Choose an input image index

xt, _ = train_loader.dataset[target_idx]
xo, _ = train_loader.dataset[input_idx]

perturbed_xo = latent_space_attack(vae, xo, xt)

# Reconstruction using the perturbed image
reconstructed_perturbed_xo, _, _ = vae(perturbed_xo)

In [ ]:
# Visualize original, target, perturbed, and reconstructed images
def show_images(original, target, perturbed, reconstructed_original, reconstructed_perturbed):
    fig, axs = plt.subplots(1, 5, figsize=(20, 6))
    
    axs[0].imshow(original.reshape(28, 28), cmap='gray')
    axs[0].set_title('Original')
    axs[0].axis('off')
    
    axs[1].imshow(target.reshape(28, 28), cmap='gray')
    axs[1].set_title('Target')
    axs[1].axis('off')
    
    axs[2].imshow(perturbed.reshape(28, 28), cmap='gray')
    axs[2].set_title('Perturbed')
    axs[2].axis('off')
    
    axs[3].imshow(reconstructed_original.reshape(28, 28), cmap='gray')
    axs[3].set_title('Reconst from Orig')
    axs[3].axis('off')
    
    axs[4].imshow(reconstructed_perturbed.reshape(28, 28), cmap='gray')
    axs[4].set_title('Reconst from Perturb')
    axs[4].axis('off')
    
    plt.tight_layout()
    plt.show()
plt.rcParams.update({'font.size': 22})


In [ ]:
reconstructed_xo, _, _ = vae(xo.unsqueeze(0))
show_images(
    xo.squeeze().detach().cpu().numpy(),
    xt.squeeze().detach().cpu().numpy(),
    perturbed_xo.squeeze().detach().cpu().numpy(),
    reconstructed_xo.squeeze().detach().cpu().numpy(),
    reconstructed_perturbed_xo.squeeze().detach().cpu().numpy()
)

In [14]:
# Function to compute reconstruction distances for each input in the dataset
def compute_reconstruction_distances(vae_model, inputs):
    distances = []
    for x in inputs:
        with torch.no_grad():
            # Encode the input x to get mean and logvar
            mu, logvar = vae_model.encode(x)
            # Reparameterize to get the latent representation z
            z = vae_model.reparameterize(mu, logvar)
            # Reconstruct the input from the latent space
            reconstructed = vae_model.decode(z)
            # Resize the reconstructed tensor to match the input size
            reconstructed = reconstructed.view(-1, 28 * 28)
            # Compute the reconstruction distance and store it
            dist = torch.dist(x.view(-1, 28 * 28), reconstructed, p=2)  # Using L2 norm
            distances.append(dist.item())
    return distances

# Function to measure r-robustness based on the computed distances
def measure_r_robustness_vae(vae_model, inputs, r):
    distances = compute_reconstruction_distances(vae_model, inputs)
    within_r = np.sum(np.array(distances) <= r)
    outside_r = len(inputs) - within_r
    within_r_prob = within_r / len(inputs)
    outside_r_prob = outside_r / len(inputs)
    return within_r_prob, outside_r_prob


data_only = [data for data, _ in train_loader.dataset]
r = 26.0  
within_r_prob, outside_r_prob = measure_r_robustness_vae(vae, data_only, r)
print(f"Probability within r: {within_r_prob}")
print(f"Probability outside r: {outside_r_prob}")


Probability within r: 0.4915
Probability outside r: 0.5085
